In [2]:
import os
os.chdir('..')

In [3]:
import torch
import numpy as np
from omegaconf import OmegaConf
from src.scoring import PrecalculationModule

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
k = torch.load('dataset/new_pharmacophores/1k1j_A.pt')

RuntimeError: Invalid magic number; corrupt file?

In [3]:
MODEL_PATH = 'model_weights/model.tar'
HEAD_PATH = 'model_weights/crossdocked_dim_128.pth'

In [4]:
head = torch.load(HEAD_PATH)

In [5]:
# predictor = PrecalculationModule(MODEL_PATH, HEAD_PATH, 'cuda')

In [6]:
score_threshold =  {
    "PiStacking_P": 0.5,
    "PiStacking_T": 0.5,
    "SaltBridge_lneg": 0.5,
    "SaltBridge_pneg": 0.5,
    "PiCation_lring": 0.5,
    "PiCation_pring": 0.5,
    "XBond": 0.5,
    "HBond_ldon": 0.5,
    "HBond_pdon": 0.5,
    "Hydrophobic": 0.5,
}

In [7]:
from src.pharmaconet.modeling import ModelingModule
module = ModelingModule('model_weights/model.tar', 'cuda',
                        score_threshold=score_threshold)

In [8]:
split_file = torch.load('dataset/pocket_to_ligands.pt')

In [9]:
for rec_id, lig_id in split_file.items():
    rec_path = os.path.join('dataset/crossdock', rec_id + '_rec.pdb')
    lig_path = os.path.join('dataset/crossdocked_pocket10', lig_id)
    
    p_model = module.run(rec_path, ref_ligand_path=lig_path)
    break

In [1]:
len(p_model.nodes)

NameError: name 'p_model' is not defined

In [9]:
# center: [ -5.1225715 -32.597176    6.1419644]

In [8]:
print(rec_id)
print(len(p_model.nodes))

4xe6_X
64
